In [ ]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import functools
import numpy as np
from scipy.optimize import curve_fit
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import Output

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt

In [ ]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [ ]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, det_diam, pulse_mode, flightPathLength_d, t0_d)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

FloatText(value=80.0, description='Detector diameter:')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

FloatText(value=54.0, description='t0:')

$$\textbf{You can specify which dataset to use in below block}$$

In [6]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_main_path = os.path.dirname(dataset_main_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/tof_calibration/'
variables.result_data_name = variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/tof_calibration/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [5]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: D:/pyccapt/tests/data/data_1658_Sep-11-2023_13-56_316L_B_17_test_1/mc_calibration
The dataset name after saving is: tof_calibration_mc_calibration_cropped_data_1658_Sep-11-2023_13-56_316L_B_17_test_1
The figures will be saved on the path: D:/pyccapt/tests/data/data_1658_Sep-11-2023_13-56_316L_B_17_test_1/mc_calibration//tof_calibration/
Total number of Ions: 198099


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,29.955000,0.0,3571.469971,535.720520,0,0.0,748.879883,-0.571429,1.306122,0,0
1,0.0,0.0,0.0,14.672785,0.0,3571.469971,535.720520,0,0.0,534.903442,0.215510,-0.940408,3885,2
2,0.0,0.0,0.0,37.790405,0.0,3571.469971,535.720520,0,0.0,825.127136,0.160000,0.313469,34,1
3,0.0,0.0,0.0,30.099127,0.0,3571.469971,535.720520,0,0.0,744.634766,0.506122,0.920816,1,1
4,0.0,0.0,0.0,14.683978,0.0,3571.469971,535.720520,0,0.0,558.769287,-0.816327,-3.673469,243,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198094,0.0,0.0,0.0,31.044279,0.0,3643.600098,546.539978,0,0.0,768.843506,1.815510,2.481633,684,2
198095,0.0,0.0,0.0,30.377861,0.0,3643.600098,546.539978,0,0.0,761.375183,1.746939,2.530612,140,1
198096,0.0,0.0,0.0,29.827755,0.0,3643.600098,546.539978,0,0.0,743.146606,2.592653,0.055510,4294951680,1
198097,0.0,0.0,0.0,29.749752,0.0,3643.600098,546.539978,0,0.0,741.857300,2.560000,0.026122,1,1


In [68]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=variables.max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(False), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
                pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='exp_hist'));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_15256\2178511811.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=variables.max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=11251.0, description='max_tof'), FloatText(value=1.0, description='frac'…

In [69]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc'), ('tof', 'tof')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=5000), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(False));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_15256\2908251488.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

$$\textbf{Below plotted graph offers peak selection. You can select relevant peaks you want for computation.}$$

In [70]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=100, description='peak prominance:')
distance=widgets.IntText(value=500, description='peak distance:')
lim_tof=widgets.IntText(value=variables.max_tof, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot peak'
)
def hist_plot(variables, plot):
    
    with out:
        clear_output(True)
    
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        
        tools.mc_hist_plot(variables, bin_size_p, mode='tof', prominence=prominence_p, distance=distance_p, percent=percent_p, selector='rect', plot=True, figname=index_fig_p, lim=lim_tof_p,
                          peaks_find_plot=plot_peak_p)

In [71]:
sample_size_b = widgets.IntText(value=11, description='sample size:') 
index_fig_b = widgets.IntText(value=1, description='fig index:')
maximum_cal_method_b = widgets.Dropdown(
    options=[('mean', 'mean'), ('histogram', 'histogram')],
    description='calib method:'
)
plot_b = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot fig:'
)

save_b = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='save fig:'
)
apply_b = widgets.Dropdown(
    options=[('all', 'all'), ('temporal', 'temporal'),],
    description='apply mode:'
)
def bowl_correction(dld_x, dld_y, dld_highVoltage, variables):
    with out:
        sample_size_p = sample_size_b.value
        index_fig_p = index_fig_b.value
        plot_p = plot_b.value
        save_p = save_b.value
        maximum_cal_method_p = maximum_cal_method_b.value
        calibration.bowl_correction_main(dld_x, dld_y, dld_highVoltage, variables, det_diam.value, sample_size=sample_size_p, maximum_cal_method=maximum_cal_method_p, apply_local=apply_b.value,
                                         calibration_mode='tof', index_fig=index_fig_p, plot=plot_p, save=save_p)


In [72]:
sample_size_v =widgets.IntText(value=100, description='sample size:') 
index_fig_v = widgets.IntText(value=1, description='fig index:')
plot_v = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot fig:'
)
save_v = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='save fig:'
)
mode_v = widgets.Dropdown(
    options=[('ion_seq', 'ion_seq'), ('voltage', 'voltage')],
    description='sample mode:'
)
peak_mode = widgets.Dropdown(
    options=[('peak', 'peak'), ('mean', 'mean'), ('median', 'median')],
    description='peak mode:'
)
num_cluster = widgets.IntText(value=1, description='num_cluster:')
apply_v = widgets.Dropdown(
    options=[('all', 'all'), ('voltage', 'voltage'), ('voltage_temporal', 'voltage_temporal')],
    description='apply mode:'
)
def vol_correction(dld_highVoltage, variables):
    with out:
        sample_size_p = sample_size_v.value
        index_fig_p = index_fig_v.value
        plot_p = plot_v.value
        save_p = save_v.value
        mode_p = mode_v.value
        peak_mode_p = peak_mode.value
        calibration.voltage_corr_main(dld_highVoltage, variables, sample_size=sample_size_p, calibration_mode='tof',
                                        index_fig=index_fig_p, plot=plot_p, save=save_p, apply_local=apply_v.value, num_cluster=num_cluster.value, mode=mode_p, peak_mode=peak_mode_p)


In [73]:
pb_bowl = widgets.HTML(
    value=" ",
    placeholder='Status:',
    description='Status:',
)
pb_vol = widgets.HTML(
    value=" ",
    placeholder='Status:',
    description='Status:',
)
plot_button = widgets.Button(
    description='plot hist',
)
plot_stat_button = widgets.Button(
    description='plot stat',
)
reset_back_button = widgets.Button(
    description='reset back correction',
)
reset_button = widgets.Button(
    description='reset',
)
save_button = widgets.Button(
    description='save correction',
)
bowl_button = widgets.Button(
    description='bowl correction',
)
vol_button = widgets.Button(
    description='voltage correction',
)
bin_fdm = widgets.IntText(value=256, description='bin FDM:')
    
@plot_button.on_click
def plot_on_click(b, variables=variables, plot=True):
    hist_plot(variables, plot)

@plot_stat_button.on_click
def plot_stat_on_click(b, variables=variables):
    with out2:
        clear_output(True)
    with out:
        clear_output(True)
        calibration.plot_selected_statistic(variables, bin_fdm.value, index_fig.value, calibration_mode='tof', save=True)
    
@reset_back_button.on_click
def reset_back_on_click(b, variables=variables):
    variables.dld_t_calib = np.copy(variables.dld_t_calib_backup)
@reset_button.on_click
def reset_on_click(b, data=data, variables=variables, flightPathLength_d=flightPathLength_d.value, t0_d=t0_d.value, max_mc=max_mc.value):
    data_tools.extract_data(data, variables, flightPathLength_d, max_mc)
    
@save_button.on_click
def save_on_click(b, variables=variables):
    variables.dld_t_calib_backup = np.copy(variables.dld_t_calib)

@vol_button.on_click
def vol_on_click(b, dld_highVoltage=variables.dld_high_voltage, variables=variables):
    with out2:
        clear_output(True)
        pb_vol.value = "<b>Starting...</b>"
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected tof ranges are: (%s, %s)' %(variables.selected_x1, variables.selected_x2))
            vol_correction(dld_highVoltage, variables)
        pb_vol.value = "<b>Flished</b>"
@bowl_button.on_click
def bowl_on_click(b, dld_x_det=variables.dld_x_det, dld_y_det=variables.dld_y_det, dld_highVoltage=variables.dld_high_voltage, variables=variables):
    with out2:
        clear_output(True)
        pb_bowl.value = "<b>Starting...</b>"
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected tof ranges are: (%s, %s)' %(variables.selected_x1, variables.selected_x2))
            bowl_correction(dld_x_det, dld_y_det, dld_highVoltage, variables)  
        pb_bowl.value = "<b>Flished</b>"


tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, bin_fdm, plot_peak])
tab2 = VBox(children=[VBox(children=[sample_size_b, index_fig_b, maximum_cal_method_b, apply_b, plot_b, save_b]), bowl_button, pb_bowl])
tab3 = VBox(children=[VBox(children=[sample_size_v, index_fig_v, mode_v, apply_v, num_cluster, peak_mode, plot_v, save_v]), vol_button, pb_vol])

tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'tof/mc plot')
tab.set_title(1, 'bowl correction')
tab.set_title(2, 'voltage correction')



display(VBox(children=[tab,HBox(children=[plot_button, plot_stat_button, save_button, reset_back_button, reset_button])]))
out = Output()
out2 = Output()
display(out)

Output()

In [ ]:
fit_result_v = [ 1.94208697e+00, -2.09506601e-04,  1.07759401e-08]
f_v = calibration.voltage_corr(variables.dld_high_voltage, *fit_result_v)
variables.dld_t_calib= variables.dld_t_calib / f_v

In [5]:
fit_result_b = [ 9.87363382e-01, -2.34910574e-04,  2.16627932e-04,  3.45115689e-05, -2.29909341e-06, 3.36282076e-05]
f_bowl = calibration.bowl_corr_fit([variables.dld_x_det*10, variables.dld_y_det*10], *fit_result_b)
variables.dld_t_calib = variables.dld_t_calib / f_bowl

In [6]:
from pyccapt.calibration.mc import mc_tools, tof_tools
tof_tools.mc2tof(1.01, 6000, 10, 10, 110)

167.4235833148962

In [49]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('tof', 'tof')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=variables.max_tof), percent=widgets.IntText(value=50), selector=fixed('peak'), plot=fixed(True), figname=widgets.Text(value='tof_hist_calibrated'),
               peaks_find_plot=fixed(True));


C:\Users\mehrp\AppData\Local\Temp\ipykernel_15256\1779110425.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('to…

In [13]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0], element[1], element[3]), "{}({})[{}]".format(element[0], element[1], element[2]))
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList,"Elements")
dropdown.observe(wd.on_change)


chargeDropdown = wd.dropdownWidget(chargeList,"Charge")
chargeDropdown.observe(wd.on_change_charge)

wd.compute_element_isotope_values_according_to_selected_charge()

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")


display(dropdown, chargeDropdown, buttonAdd, buttonDelete, buttonReset)

def buttonAdd_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickAdd(b, variables)
        display()
def buttonDelete_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickDelete(b, variables)
        display()
def buttonResett_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickReset(b, variables)
        display()

listMaterial = buttonAdd.on_click(functools.partial(buttonAdd_f, variables=variables))
buttonDelete.on_click(functools.partial(buttonDelete_f, variables=variables))
buttonReset.on_click(functools.partial(buttonResett_f, variables=variables))
# listMaterial = buttonAdd.on_click(wd.onClickAdd)
# buttonDelete.on_click(wd.onClickDelete)
# buttonReset.on_click(wd.onClickReset)
out = Output()
display(out)

Dropdown(description='Elements', options=(('H (1) (99.98)', 'H(1)[1.01]'), ('H (2) (0.01)', 'H(2)[2.01]'), ('H…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

Output()

In [14]:
peaks_chos = []
for i in range(len(variables.peaks_idx)):
    peaks_chos.append(variables.peak[variables.peaks_idx[i]])
peaks_chos = np.array(peaks_chos)
listMaterial = np.array(variables.listMaterial)
print('highest peak in the tof histogram:', peaks_chos)
print('highest peak in the ideal mc histogram:', listMaterial)

highest peak in the tof histogram: [ 226.11498269 1599.84936599 1952.88110598]
highest peak in the ideal mc histogram: [ 1.01 61.32 91.97]


In [15]:
def parametric(t, t0, c, d):

    # return c * ((t - t0)**2) + d
    return c * ((t - t0)**2) + d*t

def parametric_calib(t, mc_ideal):
    
    fitresult, _ = curve_fit(parametric, t, mc_ideal, maxfev=2000)
    return fitresult

fitresult = parametric_calib(peaks_chos, variables.listMaterial)


variables.mc_calib = parametric(variables.dld_t_calib, *fitresult)

C:\Users\mehrp\.conda\envs\apt\lib\site-packages\scipy\optimize\_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [16]:
print('The t0 is:', fitresult[0], 'ns')

The t0 is: 54.567249467574534 ns


In [17]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('peak'), plot=fixed(True), figname=widgets.Text(value='mc_hist_calibrated'),
               peaks_find_plot=fixed(True));


C:\Users\mehrp\AppData\Local\Temp\ipykernel_15256\1203954349.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [74]:
data['mc_c (Da)'] = variables.mc_calib
data['t_c (ns)'] = variables.dld_t_calib
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,96.828765,96.828765,7824.123535,43805.726562,0,1971.261651,2015.0,1.000040,-0.899989,0,0
1,0.0,0.0,0.0,97.484441,97.484441,7824.123535,43805.726562,0,1972.959243,2035.0,-0.419676,-3.137880,0,0
2,0.0,0.0,0.0,360.658488,360.658488,7824.123535,43805.726562,0,3768.501368,3877.0,-2.763780,-0.462814,0,0
3,0.0,0.0,0.0,96.552811,96.552811,7824.123535,43805.726562,0,1962.152347,2031.0,-1.613070,-3.302190,0,0
4,0.0,0.0,0.0,0.138873,0.138873,7824.123535,43805.726562,0,105.824132,108.0,0.902636,0.114006,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744458,0.0,0.0,0.0,96.236683,96.236683,7225.151367,53755.109375,0,1970.788268,2105.0,-0.585350,3.302280,0,0
744459,0.0,0.0,0.0,96.497576,96.497576,7225.151367,53755.109375,0,1970.102258,2095.0,-0.666989,1.776550,0,0
744460,0.0,0.0,0.0,96.228355,96.228355,7225.151367,53755.109375,0,1970.444166,2104.0,-0.640886,3.207970,0,0
744461,0.0,0.0,0.0,98.093664,98.093664,7225.151367,53755.109375,0,1981.190587,2108.0,-0.367809,-1.056650,0,0


In [75]:
data_c = data.copy(deep=True)

In [56]:
# Remove negative mc
threshold = 0
mc_t = data_c['mc_c (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
data_c.drop(np.where(mc_t_mask)[0], inplace=True)
data_c.reset_index(inplace=True, drop=True)

The number of ions with negative mc are: 549703


In [59]:
data_c

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.943917,-0.463124,0.019942,0.0,0.0,3682.110107,0.0,0,4311.402740,3891.519531,0.392873,0.057275,0,1
1,2.270334,-5.010604,0.551171,0.0,0.0,3682.110107,0.0,0,4321.137227,3904.508789,0.702811,-0.243931,43,1
2,-5.462508,0.471533,0.547564,0.0,0.0,3682.110107,0.0,0,4341.326190,3878.014160,-0.112540,-0.152062,211,1
3,8.305490,4.298521,1.624813,0.0,0.0,3682.110107,0.0,0,3517.824761,3204.297363,0.712028,0.788533,66,1
4,2.990205,0.078835,0.161882,0.0,0.0,3682.110107,0.0,0,3529.783961,3195.071777,0.512681,0.189050,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549698,-4.680508,-0.665500,8.134455,0.0,0.0,1334.670044,0.0,0,5695.723160,6555.320312,-0.575553,-0.705528,1,1
549699,-3.787911,-3.385010,8.331511,0.0,0.0,1334.670044,0.0,0,8418.945206,9699.115234,-0.047374,-1.282389,43,1
549700,-2.945118,-1.283401,7.480402,0.0,0.0,1334.670044,0.0,0,5753.337757,6691.295410,-0.129701,-0.605268,114,1
549701,-4.043397,-0.999465,7.856801,0.0,0.0,1334.670044,0.0,0,5724.286929,6614.916504,-0.393382,-0.694126,211,1


In [22]:
interact_manual(data_tools.save_data, data=fixed(data_c), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…

In [76]:
t = data_c['t (ns)'].to_numpy()
np.save('t_12.npy', t )
t = data_c['t_c (ns)'].to_numpy()
np.save('t_12_c.npy', t )